In [65]:
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [36]:
# define filepath
file_path = "cleaned_sentinment_1.csv"

In [37]:
sentiment_df = pd.read_csv(file_path)
sentiment_df.head()

,Unnamed: 0,ticker,date,overall_sent_comp,overall_sent_pos,overall_sent_neg,mean_t_comp_score,mean_t_pos_score,mean_t_neg_score,mean_tgt_comp_score,...,CloseOver60,VolumeOver60,CloseOver90,VolumeOver90,Co30_d,Co60_d,Co90_d,Vo30_d,Vo60_d,Vo90_d
0,0,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
1,1,DIS,2020-08-11,0.8658,0.245,0.036,0.176100,0.164667,0.0740,0.7783,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
2,2,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
3,3,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
4,4,DIS,2020-08-11,-0.5994,0.000,0.438,-0.599400,0.000000,0.4380,-0.5994,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0


In [38]:
# Data cleaning was previously performed in other files
# Set up the different target arrays
y_1 = sentiment_df['Co30_d']
y_2 = sentiment_df['Co60_d']
y_3 = sentiment_df['Co90_d']

#verb_tense = sentiment_df['verb_tense']

X = sentiment_df.drop(columns=['Close', 'ticker', 'date', 'verb_tense',
       'Volume', 'CloseOver30', 'VolumeOver30', 'CloseOver60', 'VolumeOver60',
       'CloseOver90', 'VolumeOver90', 'Co30_d', 'Co60_d', 'Co90_d', 'Vo30_d',
       'Vo60_d', 'Vo90_d'])


X = X.astype('float64')
# # add verb_tense field back in
#X['verb_tense'] = verb_tense
X = X.drop(columns='Unnamed: 0')
#X = pd.get_dummies(X)
X.head()

,overall_sent_comp,overall_sent_pos,overall_sent_neg,mean_t_comp_score,mean_t_pos_score,mean_t_neg_score,mean_tgt_comp_score,mean_tgt_pos_score,mean_tgt_neg_score,mean_custom_score
0,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,0.084,0.157,0.0
1,0.8658,0.245,0.036,0.176100,0.164667,0.0740,0.7783,0.494,0.000,0.0
2,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,0.084,0.157,0.0
3,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,0.084,0.157,0.0
4,-0.5994,0.000,0.438,-0.599400,0.000000,0.4380,-0.5994,0.000,0.438,0.0


### Logistic Regression Models

In [39]:
# Split the preprocessed data into a training and testing dataset for N30d
X_train, X_test, y_train, y_test = train_test_split(X, y_1)

In [40]:

X_train.head()

,overall_sent_comp,overall_sent_pos,overall_sent_neg,mean_t_comp_score,mean_t_pos_score,mean_t_neg_score,mean_tgt_comp_score,mean_tgt_pos_score,mean_tgt_neg_score,mean_custom_score
269833,-0.3559,0.076,0.085,-0.07220,0.081667,0.084000,0.4404,0.245,0.000,0.0
155364,0.7983,0.025,0.011,0.79830,0.025000,0.011000,0.7983,0.025,0.011,1.0
691694,-0.6705,0.125,0.213,-0.07975,0.213000,0.168500,-0.7964,0.000,0.337,-1.0
173030,0.5195,0.024,0.018,0.07180,0.008333,0.132333,0.7983,0.025,0.011,1.0
55496,0.8758,0.030,0.011,0.87580,0.030000,0.011000,0.8758,0.030,0.011,1.0


In [41]:
# Train the Logistic Regression model 
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [42]:
# Calculated the balanced accuracy score for N30Days
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5795652588378268

In [43]:
# Split the preprocessed data into a training and testing dataset for N60d
X_train, X_test, y_train, y_test = train_test_split(X, y_2)

In [44]:
# Train the Logistic Regression model 
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [45]:
# Calculated the balanced accuracy score for N60Days
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5008455425272363

In [46]:
# Split the preprocessed data into a training and testing dataset for N90d
X_train, X_test, y_train, y_test = train_test_split(X, y_3)

In [47]:
# Train the Logistic Regression model 
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [48]:
# Calculated the balanced accuracy score for N90Days
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

### Balanced Random Forest Classifier

In [52]:
# Split the preprocessed data into a training and testing dataset for N30d
X_train, X_test, y_train, y_test = train_test_split(X, y_1)

In [53]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100)  # 100 estimators is default
rf_model

BalancedRandomForestClassifier(random_state=1)

In [54]:
# Calculated the balanced accuracy score
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6531089749482861

In [56]:
# Split the preprocessed data into a training and testing dataset for N60d
X_train, X_test, y_train, y_test = train_test_split(X, y_2)

In [57]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100)  # 100 estimators is default
rf_model

BalancedRandomForestClassifier(random_state=1)

In [58]:
# Calculated the balanced accuracy score
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6130188716840952

In [59]:
# Split the preprocessed data into a training and testing dataset for N90d
X_train, X_test, y_train, y_test = train_test_split(X, y_3)

In [60]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100)  # 100 estimators is default
rf_model

BalancedRandomForestClassifier(random_state=1)

In [61]:
# Calculated the balanced accuracy score
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6055138596316851

In [62]:
rf_model.feature_importances_

array([0.1306491 , 0.11635813, 0.09335004, 0.13167919, 0.10860028,
       0.08933545, 0.11686865, 0.09497116, 0.07004776, 0.04814025])

In [63]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.13167918718135, 'mean_t_comp_score'),
 (0.13064909723728418, 'overall_sent_comp'),
 (0.11686865452567494, 'mean_tgt_comp_score'),
 (0.11635812920233551, 'overall_sent_pos'),
 (0.1086002752244493, 'mean_t_pos_score'),
 (0.09497115707710754, 'mean_tgt_pos_score'),
 (0.09335003604714269, 'overall_sent_neg'),
 (0.0893354547197863, 'mean_t_neg_score'),
 (0.07004775715321962, 'mean_tgt_neg_score'),
 (0.048140251631649915, 'mean_custom_score')]

### Easy Ensemble AdaBoost Classifier

In [66]:
# Train the EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100)

In [67]:
# Split the preprocessed data into a training and testing dataset for N30d
X_train, X_test, y_train, y_test = train_test_split(X, y_1)

In [68]:
# Calculated the balanced accuracy score
ee_model = ee_model.fit(X_train, y_train)
predictions = ee_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.5888230948451443

In [69]:
# Split the preprocessed data into a training and testing dataset for N60d
X_train, X_test, y_train, y_test = train_test_split(X, y_2)

In [70]:
# Calculated the balanced accuracy score
ee_model = ee_model.fit(X_train, y_train)
predictions = ee_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.5395573863334435

In [71]:
# Split the preprocessed data into a training and testing dataset for N90d
X_train, X_test, y_train, y_test = train_test_split(X, y_3)

In [72]:
# Calculated the balanced accuracy score
ee_model = ee_model.fit(X_train, y_train)
predictions = ee_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.5221864124308839